# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [19]:
activity=3987156049
LAST=True

In [20]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [21]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [22]:
sgmt

,Name,Time,Speed,Power,Info
0,Puente 77 Sur 0.72km 14m 1%,2:50,15.4km/h,200W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
1,Puente 77 Sur - Variante 0.75km 29m 4%,3:00,15.0km/h,285W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
2,Union - Aries 0.54km 26m 4%,2:22,13.8km/h,275W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
3,Unicornio - Monaco 2.28km 85m 3%,8:55,15.3km/h,228W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
4,unicornio ancon 1.13km 63m 5%,5:25,12.6km/h,245W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
5,Sprint Largo (El ciclista) 0.61km 22m 3%,2:59,12.4km/h,235W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
6,Sprint (El ciclista) 0.34km 17m 5%,1:49,11.3km/h,241W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
7,Ancón Sur - Caldas 6.22km 88m 1%,19:30,19.2km/h,218W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
8,La Estrella Climb 0.86km 33m 4%,3:43,14.0km/h,254W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."
9,Inmaculada - Caldas 4.37km 44m 1%,13:28,19.5km/h,215W,"{'Calories': {'Avg': '1095', 'Max': '1095'}, '..."


In [23]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [24]:
df.to_json(db_file)

### Save notebook here

In [25]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [26]:
!bash commit.sh

[master 3a9bed9] Morning Ride 9:11 AM on Sunday, August 30, 2020
 2 files changed, 228 insertions(+), 409 deletions(-)
 rewrite Strava.ipynb (60%)
 rewrite db/strava.json (69%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 7.86 KiB | 804.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   efc8147..3a9bed9  master -> master


In [27]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
